In [13]:
import os 
import pandas as pd
import numpy as np
import arcpy


In [14]:
# Enable overwriting of outputs
arcpy.env.overwriteOutput = True

input_shapefile = r"C:\Users\Sea Grant\Desktop\WW_Overlay\MauiTMK\MauiTMK.shp"

workspace = os.path.dirname(input_shapefile)
arcpy.env.workspace = workspace


# Output temporary layers
temp_output = "in_memory\\temp_output"
# Delete temp_output if it exists
if arcpy.Exists(temp_output):
    arcpy.management.Delete(temp_output)

# Create a copy of the input layer to avoid modifying the original
arcpy.management.CopyFeatures(input_shapefile, temp_output)


# Field names 
tax_plat_field = "TaxPlat"   # Field that groups records by Tax Plat
sewer_field = "Laterals"        # Field indicating if the TMK intersects a sewer (e.g., "Yes")
unique_field = "TMK"    # Unique identifier for each TMK record (change if necessary)

# Field for sewer intersection count
sewer_num_field = "SewerNum"


In [15]:

# Add and Calculate a Numeric Field for Sewer Intersection

# Check if 'SewerNum' already exists; if not, add it as a short integer
fields = [f.name for f in arcpy.ListFields(input_shapefile)]
if sewer_num_field not in fields:
    arcpy.AddField_management(in_table=input_shapefile,
                              field_name=sewer_num_field,
                              field_type="SHORT")
else:
    arcpy.AddMessage(f"Field '{sewer_num_field}' already exists.")

# Calculate the 'SewerNum' field: set to 1 if 'Sewer' equals "Yes"; otherwise, set to 0.
calc_expression = "1 if !{}! == 'Yes' else 0".format(sewer_field)
arcpy.CalculateField_management(in_table=input_shapefile,
                                field=sewer_num_field,
                                expression=calc_expression,
                                expression_type="PYTHON3")
arcpy.AddMessage(f"Calculated field '{sewer_num_field}' based on '{sewer_field}' values.")


In [16]:

#  Summarize Data by Tax Plat
summary_table = r"in_memory\TaxPlat_SewerSummary"

# Define statistics:
#   - COUNT of the unique_field gives the total number of TMKs per Tax Plat.
#   - SUM of the SewerNum field gives the number of TMKs intersecting a sewer.
statistics_fields = [[unique_field, "COUNT"], [sewer_num_field, "SUM"]]

arcpy.analysis.Statistics(in_table=input_shapefile,
                          out_table=summary_table,
                          statistics_fields=statistics_fields,
                          case_field=[tax_plat_field])

In [17]:

# Calculate the Percentage of TMKs Intersecting a Sewer

# Add a new field to the summary table for the percentage
perc_field = "SewerPerc"
arcpy.AddField_management(in_table=summary_table,
                          field_name=perc_field,
                          field_type="DOUBLE")

# The Statistics tool automatically names the output fields as follows:
#   - "COUNT_<unique_field>" for the count of TMKs.
#   - "SUM_<sewer_num_field>" for the sum of sewer intersections.
count_field = "COUNT_" + unique_field
sum_field   = "SUM_" + sewer_num_field

# Define a code block that calculates the percentage.
# This function checks if the count is zero to avoid division by zero.
code_block = """
def calcPerc(count_val, sum_val):
    if count_val == 0:
        return 0
    else:
        return (sum_val / count_val) * 100
"""

# Use the code block in the Calculate Field tool.
expression = "calcPerc(!{0}!, !{1}!)".format(count_field, sum_field)

arcpy.CalculateField_management(in_table=summary_table,
                                field=perc_field,
                                expression=expression,
                                expression_type="PYTHON3",
                                code_block=code_block)


In [18]:
#Join the Summary Table Back to the Input Shapefile
# -------------------------------
# This join adds the calculated SewerPerc field (from the summary table) to each TMK record.
arcpy.JoinField_management(in_data=input_shapefile,
                           in_field=tax_plat_field,
                           join_table=summary_table,
                           join_field=tax_plat_field,
                           fields=[perc_field])

# To ensure the join is permanent, make a copy of the joined features.
joined_fc = os.path.join(workspace, "TMK_TaxPlat_Joined.shp")
arcpy.CopyFeatures_management(in_features=input_shapefile, out_feature_class=joined_fc)

out_feature_class = r"C:\Users\Sea Grant\Desktop\WW_Overlay\MauiTMK\MauiTMK_LatPCT.shp"

ExecuteError: ERROR 160196: Invalid Topology
Failed to execute (Dissolve).
